# Text Generation with RNNs


In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import random

### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
import urllib  # the lib that handles the url stuff

text = ""

for line in urllib.request.urlopen("https://raw.githubusercontent.com/GITenberg/Moby-Dick--Or-The-Whale_2701/master/2701.txt"):
    text += line.decode('utf-8')


#### Inspect the dataset
Take a look at the first 250 characters in text

In [3]:
print(text)

The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Moby Dick; or The Whale

Author: Herman Melville

Last Updated: January 3, 2009
Posting Date: December 25, 2008 [EBook #2701]
Release Date: June, 2001

Language: English

Character set encoding: ASCII

*** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE ***




Produced by Daniel Lazarus and Jonesey





MOBY DICK; OR THE WHALE

By Herman Melville




Original Transcriber's Notes:

This text is a combination of etexts, one from the now-defunct ERIS
project at Virginia Tech and one from Project Gutenberg's archives. The
proofreaders of this version are indebted to The University of Adelaide
Library for preserving the Virginia Tech version. The r

In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [6]:

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [7]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))



'The Project G' ---- characters mapped to int ---- > [50 67 64  2 46 77 74 69 64 62 79  2 37]


#### Defining a method to encode one hot labels

In [8]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((len(arr), n_labels))
    for i, el in enumerate(arr):
        one_hot[i][el] = 1
    return one_hot


#### Defining a method to make mini-batches for training

In [9]:
def get_batches(arr, batch_size, seq_length, vocab_len):
    '''Creates a generator that returns batches of size
       (batch_size, vocab_len), (batch_size, vocab_len) from arr.

       Arguments
       ---------
       arr: Array of encoded words you want to make batches from.
       batch_size: Batch size, size of the tuples which are returned.
       seq_length: Number of encoded chars in a sequence
    '''
    x_batch = []
    y_batch = []
    start_idxs = range(0, len(arr) - seq_length - 1, seq_length)
    for start_idx in start_idxs:
        x = one_hot_encode(arr[start_idx:start_idx + seq_length], vocab_len)
        y = arr[start_idx + 1:start_idx + seq_length + 1]
        x_batch.append(x)
        y_batch.append(y)
        if len(x_batch) == batch_size:
            yield x_batch, y_batch
            x_batch = []
            y_batch = []
    if len(x_batch) > 0:
        yield x_batch, y_batch


## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda



### Declaring the model

In [11]:
class VanillaRNNLayer(nn.Module):
    def __init__(self, x_size, a_size, y_size, is_last):
        super().__init__()
        self.w_ax = nn.Linear(x_size, a_size)
        self.w_aa = nn.Linear(a_size, a_size, bias=False)
        self.w_ya = nn.Linear(a_size, y_size)
        self.a_size = a_size
        self.is_last = is_last
        
    def forward(self, x, last_a):
        '''Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        a = nn.functional.relu(self.w_aa(last_a) + self.w_ax(x))
        out = self.w_ya(a)
        if not self.is_last:
            out = nn.functional.relu(out)
        return out, a

class VanillaRNN(nn.Module):
    def __init__(self, x_size, a_size, y_size, n_layers):
        super().__init__()
        # Either w_ax or w_aa doesn't need a bias
        layers = []
        for layer in range(n_layers):
            is_last = layer == (n_layers - 1)
            layers.append(VanillaRNNLayer(x_size, a_size, y_size, is_last=is_last))
        self.layers = torch.nn.ModuleList(layers)
        self.a_size = a_size
        
    def forward(self, x, last_h_states):
        '''Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        next_h_states = []
        for hidden_state, layer in zip(last_h_states, self.layers):
            x, next_h_state = layer(x, hidden_state)
            next_h_states.append(next_h_state)
        return x, next_h_states

    def init_hidden(self):
        ''' Initializes hidden state '''
        a = [torch.zeros((1, self.a_size)).to(device) for i in range(len(self.layers))]
        return a

In [12]:
class VanillaGRULayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, y_size, is_last):
        super().__init__()
        self.w_z = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.w_r = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.w_h_tilde = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.w_ya = nn.Linear(hidden_dim, y_size)
        self.hidden_dim = hidden_dim
        self.is_last = is_last
        
    def forward(self, x, last_a):
        '''Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        z_t = nn.functional.sigmoid(self.w_z(torch.cat([last_a, x], dim=1)))
        r_t = nn.functional.sigmoid(self.w_r(torch.cat([last_a, x], dim=1)))
        h_tilde = nn.functional.tanh(self.w_r(torch.cat([r_t * last_a, x], dim=1)))
        h_t = (1 - z_t) * last_a + z_t * h_tilde
        out = self.w_ya(h_t)
        if not self.is_last:
            out = nn.functional.relu(out)
        return out, h_t

class VanillaGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, y_size, n_layers):
        super().__init__()
        # Either w_ax or w_aa doesn't need a bias
        layers = []
        for layer in range(n_layers):
            is_last = layer == (n_layers - 1)
            layers.append(VanillaGRULayer(input_dim, hidden_dim, y_size, is_last=is_last))
        self.layers = torch.nn.ModuleList(layers)
        self.hidden_dim = hidden_dim
        
    def forward(self, x, last_h_states):
        '''Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        next_h_states = []
        for hidden_state, layer in zip(last_h_states, self.layers):
            x, next_h_state = layer(x, hidden_state)
            next_h_states.append(next_h_state)
        return x, next_h_states

    def init_hidden(self):
        ''' Initializes hidden state '''
        a = [torch.zeros((1, self.hidden_dim)).to(device) for i in range(len(self.layers))]
        return a


#### Declaring the train method

In [13]:
def train(model, data, vocab_len, epochs=10, seq_length=50, lr=0.0005, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    '''

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    train_losses = []
    val_losses = []
    for e in range(epochs):
        running_train_losses = []
        running_val_losses = []
        h = model.init_hidden()
        
        model.train()
        for x_batch, y_batch in tqdm(get_batches(data, batch_size, seq_length, vocab_len)):
            for x_sequence, y_sequence in zip(x_batch, y_batch):
                # disconnect gradient computation
                h = [el.data for el in h]
                opt.zero_grad()
                outputs = []
                
                for inp_char in x_sequence:
                    inp_char = torch.FloatTensor([inp_char]).to(device)
                    out, h = model(inp_char, h)
                    outputs.append(out)
                
                outputs = torch.stack(outputs, dim=0).squeeze(dim=1)
                y_sequence = torch.LongTensor(y_sequence).to(device)
                loss = criterion(outputs, y_sequence)
                loss.backward()
                opt.step()
                running_train_losses.append(loss.item())
        
        model.eval()
        h = model.init_hidden()
        with torch.no_grad():
            for x_batch, y_batch in tqdm(get_batches(data, batch_size, seq_length, vocab_len)):
                for x_sequence, y_sequence in zip(x_batch, y_batch):
                    outputs = []
                    for inp_char in x_sequence:
                        inp_char = torch.FloatTensor([inp_char]).to(device)
                        out, h = model(inp_char, h)
                        outputs.append(out)

                    outputs = torch.stack(outputs, dim=0).squeeze(dim=1)
                    y_sequence = torch.LongTensor(y_sequence).to(device)
                    loss = criterion(outputs, y_sequence)
                    running_val_losses.append(loss.item())

        train_loss = np.mean(np.array(running_train_losses))
        val_loss = np.mean(np.array(running_val_losses))
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"Epoch {e + 1}/{epochs} => train_loss: {train_loss}, val_loss: {val_loss}")
    
    return train_losses, val_losses


##### Defining a method to generate the next character

In [14]:
def predict(model, char, vocab_len, h, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, vocab_len)
    inputs = torch.from_numpy(x).float().to(device)

    # detach hidden state from history
    h = [each.data.float() for each in h]
    with torch.no_grad():
        output, h = model(inputs, h)
    
    output = nn.functional.softmax(output, dim=1)
    p = output.cpu().numpy().squeeze()
    p_sorted = np.sort(p)
    p = np.where(p < p_sorted[-top_k], np.zeros_like(p), p)
    # make it sum to one again
    p = p / np.sum(p)
    char = np.random.choice(np.array(list(range(vocab_len))), p=p)
    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [15]:
def sample(model, size, vocab_len, prime='The whale is attacking ', top_k=3):
    model.eval()

    chars = [ch for ch in prime]
    h = model.init_hidden()
    for ch in prime:
        char, h = predict(model, ch, vocab_len, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
        char, h = predict(model, char, vocab_len, h, top_k=top_k)
        chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [16]:
vanilla_model = VanillaRNN(x_size=len(vocab), a_size=256, y_size=len(vocab), n_layers=2).to(device)
print(vanilla_model)
gru_model = VanillaGRU(input_dim=len(vocab), hidden_dim=256, y_size=len(vocab), n_layers=2).to(device)
print(gru_model)

VanillaRNN(
  (layers): ModuleList(
    (0): VanillaRNNLayer(
      (w_ax): Linear(in_features=86, out_features=256, bias=True)
      (w_aa): Linear(in_features=256, out_features=256, bias=False)
      (w_ya): Linear(in_features=256, out_features=86, bias=True)
    )
    (1): VanillaRNNLayer(
      (w_ax): Linear(in_features=86, out_features=256, bias=True)
      (w_aa): Linear(in_features=256, out_features=256, bias=False)
      (w_ya): Linear(in_features=256, out_features=86, bias=True)
    )
  )
)
VanillaGRU(
  (layers): ModuleList(
    (0): VanillaGRULayer(
      (w_z): Linear(in_features=342, out_features=256, bias=True)
      (w_r): Linear(in_features=342, out_features=256, bias=True)
      (w_h_tilde): Linear(in_features=342, out_features=256, bias=True)
      (w_ya): Linear(in_features=256, out_features=86, bias=True)
    )
    (1): VanillaGRULayer(
      (w_z): Linear(in_features=342, out_features=256, bias=True)
      (w_r): Linear(in_features=342, out_features=256, bias=True


###### Declaring the hyperparameters

In [17]:
batch_size = 128
seq_length = 30
n_epochs = 2


##### Train the model and have fun with the generated texts

In [18]:
van_train_losses, van_val_losses = train(vanilla_model, text_as_int, vocab_len=len(vocab), epochs=n_epochs, seq_length=seq_length, lr=0.001, print_every=50)

295it [20:58,  4.26s/it]
295it [07:19,  1.49s/it]
0it [00:00, ?it/s]

Epoch 1/2 => train_loss: 1.7786692310110908, val_loss: 1.6848455361737285


295it [21:40,  4.41s/it]
295it [07:20,  1.49s/it]

Epoch 2/2 => train_loss: 1.5969209847658468, val_loss: 1.6273821058687714


In [19]:
van_train_losses, van_val_losses = train(gru_model, text_as_int, vocab_len=len(vocab), epochs=n_epochs, seq_length=seq_length, lr=0.001, print_every=50)

0it [00:00, ?it/s]/home/tim/.local/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/tim/.local/lib/python3.8/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
295it [42:05,  8.56s/it]
295it [15:40,  3.19s/it]
0it [00:00, ?it/s]

Epoch 1/2 => train_loss: 1.7561939787348266, val_loss: 1.6541441834898314


295it [42:24,  8.62s/it]
295it [15:36,  3.17s/it]

Epoch 2/2 => train_loss: 1.566473465278832, val_loss: 1.5933792552142334


In [20]:
print(sample(vanilla_model, 600, len(vocab)))

The whale is attacking of the should born and sea, as the sea to their concluding his been stare to sterns to sometime that
heart of the concerning here of the
stretging and so that have a strength, they was thin in this would to that a considering her carpent and that he had been stern that there, they was
stare a hold the ship's seated to the casks of their seater that heaven, when," said the ship and so and this see of that standing the sea and a sea and they say on the ship was a seat and the long see a consideral sea too with the concernation of
that and souls, though that
and all the crew, when there, with


In [21]:
print(sample(gru_model, 600, len(vocab)))

The whale is attacking to
the fing--they saw their ship was seals wild
but in the sea."

There were stood when
this shark one witcher, they had new sound and they high fishing the fired before as the four considing to her
with sort of the first was no strines they were supers the state.

"They said a thing and seemed that to the starbs, the stern things of the
strust of the founded was as they heard, as if ye cried whales and stern by this, as they, and that the othing way at the curision
at lest it were sunding to be from the fine. But were superstory with the state as the fear to tent officer, and
the wi
